In [1]:
# Training script for LazyConfig models
try:
    # ignore ShapelyDeprecationWarning from fvcore
    import warnings

    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

except:
    pass
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger

setup_logger()

import gc
import os
import time

import detectron2.utils.comm as comm

# import some common libraries
import numpy as np
import torch

# import some common detectron2 utilities
from detectron2.config import LazyConfig, get_cfg
import detectron2.data as data
from detectron2.engine import (
    launch,
)

from deepdisc.data_format.augment_image import train_augs
from deepdisc.data_format.image_readers import DC2ImageReader
from deepdisc.data_format.register_data import (
    register_data_set,
)  # , register_loaded_data_set
from deepdisc.model.loaders import (
    RedshiftFlatDictMapper,
    return_test_loader,
    return_train_loader,
)
from deepdisc.model.models import (
    RedshiftPointCasROIHeads,
    RedshiftPointROIHeads,
    RedshiftPDFROIHeads,
    return_lazy_model,
)
from deepdisc.training.trainers import (
    return_evallosshook,
    return_lazy_trainer,
    return_optimizer,
    return_savehook,
    return_schedulerhook,
)
from deepdisc.utils.parse_arguments import make_training_arg_parser
from deepdisc.inference.predictors import return_predictor_transformer

In [2]:
# from rail.estimation.algos.deepdisc import DeepDiscInformer
import rail
from rail.estimation.algos.deepdisc import *
from rail.core.data import TableHandle, JsonHandle
from rail.core.stage import RailStage



In [3]:
#cfgfile = "./tests/configs/solo/solo_res50_DC2.py"
cfgfile = "./configs/solo/solo_swin_DC2.py"

output_dir = "./"
output_name = "test"

trainfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_train.hdf5"
testfile = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test.hdf5"
metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/train_metadata.hdf5"
test_metadatafile = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata.hdf5"


In [4]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [5]:
training = DS.add_data(
    "training", data=None, handle_class=TableHandle, path=trainfile
)  # ()["images"]
testing = DS.add_data("testing", data=None, handle_class=TableHandle, path=testfile)

#metadatahandle = DS.add_data("metadata", metadata, JsonHandle, path=metadatafile)
metadatahandle = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile)

In [6]:
#test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
#test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
#metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
#metadatahandle_with_chunks = DS.add_data("metadata", metadata, Hdf5Handle, path=metadatafile_with_chunks)

In [7]:
deep_dict = dict(
    chunk_size=5,
    epoch=20,
    batch_size=1,
    output_dir="./",
    cfgfile = cfgfile,
    num_gpus=1,
    print_frequency=5,
    head_epochs=1,
    full_epochs=1,
)

In [8]:
Inform = DeepDiscInformer.make_stage(
    name="Inform_DeepDISC", model="test_informer.pkl", **deep_dict
)

In [ ]:
# Inform.inform(training, metadatahandle)
Inform.inform(training, metadatahandle) 

### Inference


In [ ]:
# metadatahandle = DS.add_data(
#     "metadata", test_metadata, JsonHandle, path=test_metadatafile
# )

In [ ]:
# Estimator = DeepDiscPDFEstimator.make_stage(
#     name="DeepDiscEstimator",
#     model=Inform.get_handle("model"),
#     # hdf5_groupname="images",
#     **deep_dict,
# )

In [ ]:
# Estimator.config

In [ ]:
# results = Estimator.estimate(testing, metadatahandle)

In [ ]:
# res = results.read()

In [ ]:
# truth = Estimator.get_handle("truth")

In [ ]:
# ztrue = truth.data["redshift"]

In [8]:
from rail.core.data import ModelHandle

In [9]:
MH = ModelHandle('model',path='./test_informer.pkl')
data = MH.read()
#MH()

In [9]:
cfgfile = "./configs/solo/solo_swin_DC2.py"
cfg = LazyConfig.load(cfgfile)

In [10]:
#MH()

In [16]:
checkpoint = MH()['nnmodel']

In [ ]:
#checkpoint=Inform.get_handle("model")
#help(checkpoint)

In [17]:
#predictor = return_predictor_transformer(cfg, checkpoint=checkpoint)
import deepdisc.astrodet.astrodet as toolkit
predictor = toolkit.AstroPredictor(cfg,checkpoint=checkpoint)

checkpoint


## Test new chunking algorithm

In [10]:
deep_estimation_dict = dict(
    chunk_size=5,
    output_dir="./",
    cfgfile = cfgfile,
    zmin=0,
    zmax=5,
    nzbins=200,
    output_mode='default',
)

EstimatorWithChunks = DeepDiscPDFEstimatorWithChunking.make_stage(
    name="DeepDiscEstimatorWithChunks",
    #model=Inform.get_handle("model"),
    model=MH,
    **deep_estimation_dict,
)

In [11]:
test_file_for_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/flattened_images_test_small.hdf5"
test_handle_for_chunks = DS.add_data("testing", data=None, handle_class=TableHandle, path=test_file_for_chunks)
metadatafile_with_chunks = "/home/shared/hsc/DC2/test_data/dataset_3/test_metadata_example.hdf5"
metadatahandle_with_chunks = DS.add_data("metadata", data=None, handle_class=Hdf5Handle, path=metadatafile_with_chunks)

In [12]:
results_from_chunks = EstimatorWithChunks.estimate(test_handle_for_chunks, metadatahandle_with_chunks)

Inserting handle into data store.  model: ./test_informer.pkl, DeepDiscEstimatorWithChunks
Caching data
Processing chunk (start:end) - (0:5)
Matching objects
[01/09 21:04:19 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
Matching objects
[01/09 21:04:19 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/09 21:04:19 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/09 21:04:19 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[01/09 21:04:19 d2.data.common]: Serialized dataset takes 1.29 MiB
[01/09 21:04:19 d2.data.common]: Serialized dataset takes 1.29 MiB


/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /data/miniconda3/envs/opence-1.7/conda-bld/pytorch-base_1672876060819/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ProcessRaisedException: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/g4merz/detectron2/detectron2/engine/launch.py", line 123, in _distributed_worker
    main_func(*args)
  File "/home/g4merz/rail_deepdisc/src/rail/estimation/algos/deepdisc.py", line 410, in _do_inference
    dist.gather(pdfs, gather_list=[], dst=0, group=group)
  File "/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/distributed/distributed_c10d.py", line 2513, in gather
    _check_single_tensor(tensor, "tensor")
  File "/home/g4merz/.conda/envs/ddrailnv/lib/python3.9/site-packages/torch/distributed/distributed_c10d.py", line 463, in _check_single_tensor
    raise RuntimeError(
RuntimeError: Invalid function argument. Expected parameter `tensor` to be of type torch.Tensor.


In [ ]:
res_ens = results_from_chunks.read()
res_ens.npdf

In [ ]:
res_ens

In [20]:
metadata_iterator = EstimatorWithChunks.input_iterator("metadata")

In [21]:
_,_,metadata_json_dicts = next(metadata_iterator)

In [26]:
print(metadata_json_dicts.keys())

odict_keys(['metadata_dicts'])


In [27]:
metadata = [json.loads(this_json) for this_json in metadata_json_dicts['metadata_dicts']]


In [ ]:
#print(metadata[0])

## Evaluation

In [1]:
from rail.evaluation.evaluator import Evaluator

In [2]:
deep_eval_dict = dict(
    chunk_size=100,
    zmin=-1,
    zmax=5,
    nzbins=200,
    epochs=20,
    output_name="test_evaluator",
    point_metrics="",
)

In [3]:
DeepEvaluator = Evaluator.make_stage(name="DeepDiscEvaluator", **deep_eval_dict)

In [4]:
eval_res = DeepEvaluator.evaluate(res, truth)

NameError: name 'res' is not defined

In [ ]:
eval_res.data

In [ ]:
from qp.metrics.pit import PIT
from utils import *  # plot_pit_qq, ks_plot

pitobj = PIT(res, ztrue)
pit_out_rate = pitobj.evaluate_PIT_outlier_rate()

In [ ]:
from qp.ensemble import Ensemble


class Sample(Ensemble):
    """Expand qp.Ensemble to append true redshifts
    array, metadata, and specific plots."""

    def __init__(
        self, pdfs, zgrid, ztrue, photoz_mode=None, code="", name="", n_quant=100
    ):
        """Class constructor

        Parameters
        ----------
        pdfs: `ndarray`
            photo-z PDFs array, shape=(Ngals, Nbins)
        zgrid: `ndarray`
            PDF bins centers, shape=(Nbins,)
        ztrue: `ndarray`
            true redshifts, shape=(Ngals,)
        photoz_mode: `ndarray`
            photo-z (PDF mode), shape=(Ngals,)
        code: `str`, (optional)
            algorithm name (for plot legends)
        name: `str`, (optional)
            sample name (for plot legends)
        """

        super().__init__(interp, data=dict(xvals=zgrid, yvals=pdfs))
        self._pdfs = pdfs
        self._zgrid = zgrid
        self._ztrue = ztrue
        self._photoz_mode = photoz_mode
        self._code = code
        self._name = name
        self._n_quant = n_quant
        self._pit = None
        self._qq = None

    @property
    def code(self):
        """Photo-z code/algorithm name"""
        return self._code

    @property
    def name(self):
        """Sample name"""
        return self._name

    @property
    def ztrue(self):
        """True redshifts array"""
        return self._ztrue

    @property
    def zgrid(self):
        """Redshift grid (binning)"""
        return self._zgrid

    @property
    def photoz_mode(self):
        """Photo-z (mode) array"""
        return self._photoz_mode

    @property
    def n_quant(self):
        return self._n_quant

    @property
    def pit(self):
        if self._pit is None:
            pit_array = np.array(
                [self[i].cdf(self.ztrue[i])[0][0] for i in range(len(self))]
            )
            self._pit = pit_array
        return self._pit

    @property
    def qq(self, n_quant=100):
        q_theory = np.linspace(0.0, 1.0, n_quant)
        q_data = np.quantile(self.pit, q_theory)
        self._qq = (q_theory, q_data)
        return self._qq

    def __len__(self):
        if len(self._ztrue) != len(self._pdfs):
            raise ValueError("Number of pdfs and true redshifts do not match!!!")
        return len(self._ztrue)

    def __str__(self):
        code_str = f"Algorithm: {self._code}"
        name_str = f"Sample: {self._name}"
        line_str = "-" * (max(len(code_str), len(name_str)))
        text = str(
            line_str
            + "\n"
            + name_str
            + "\n"
            + code_str
            + "\n"
            + line_str
            + "\n"
            + f"{len(self)} PDFs with {len(self.zgrid)} probabilities each \n"
            + f"qp representation: {self.gen_class.name} \n"
            + f"z grid: {len(self.zgrid)} z values from {np.min(self.zgrid)} to {np.max(self.zgrid)} inclusive"
        )
        return text

    def plot_pdfs(self, gals, show_ztrue=True, show_photoz_mode=False):
        colors = plot_pdfs(
            self, gals, show_ztrue=show_ztrue, show_photoz_mode=show_photoz_mode
        )
        return colors

    def plot_old_valid(self, gals=None, colors=None):
        old_metrics_table = plot_old_valid(self, gals=gals, colors=colors)
        return old_metrics_table

    def plot_pit_qq(
        self,
        bins=None,
        label=None,
        title=None,
        show_pit=True,
        show_qq=True,
        show_pit_out_rate=True,
        savefig=False,
    ):
        """Make plot PIT-QQ as Figure 2 from Schmidt et al. 2020."""
        fig_filename = plot_pit_qq(
            self,
            bins=bins,
            label=label,
            title=title,
            show_pit=show_pit,
            show_qq=show_qq,
            show_pit_out_rate=show_pit_out_rate,
            savefig=savefig,
        )
        return fig_filename

In [ ]:
def plot_pit_qq(
    pdfs,
    zgrid,
    ztrue,
    bins=None,
    title=None,
    code=None,
    show_pit=True,
    show_qq=True,
    pit_out_rate=None,
    savefig=False,
) -> str:
    """Quantile-quantile plot
        Ancillary function to be used by class Metrics.
    ​
        Parameters
        ----------
        pit: `PIT` object
            class from metrics.py
        bins: `int`, optional
            number of PIT bins
            if None, use the same number of quantiles (sample.n_quant)
        title: `str`, optional
            if None, use formatted sample's name (sample.name)
        label: `str`, optional
            if None, use formatted code's name (sample.code)
        show_pit: `bool`, optional
            include PIT histogram (default=True)
        show_qq: `bool`, optional
            include QQ plot (default=True)
        pit_out_rate: `ndarray`, optional
            print metric value on the plot panel (default=None)
        savefig: `bool`, optional
            save plot in .png file (default=False)
    """

    if bins is None:
        bins = 100
    if title is None:
        title = ""

    if code is None:
        code = ""
        label = ""
    else:
        label = code + "\n"

    if pit_out_rate is not None:
        try:
            label += "PIT$_{out}$: "
            label += f"{float(pit_out_rate):.4f}"
        except:
            print("Unsupported format for pit_out_rate.")

    plt.figure(figsize=[4, 5])
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
    ax0 = plt.subplot(gs[0])
    sample = Sample(pdfs, zgrid, ztrue)

    if show_qq:
        ax0.plot(
            sample.qq[0], sample.qq[1], c="r", linestyle="-", linewidth=3, label=label
        )
        ax0.plot([0, 1], [0, 1], color="k", linestyle="--", linewidth=2)
        ax0.set_ylabel("Q$_{data}$", fontsize=18)
        plt.ylim(-0.001, 1.001)
    plt.xlim(-0.001, 1.001)
    plt.title(title)
    if show_pit:
        fzdata = Ensemble(interp, data=dict(xvals=zgrid, yvals=pdfs))
        pitobj = PIT(fzdata, ztrue)
        pit_vals = np.array(pitobj.pit_samps)
        pit_out_rate = pitobj.evaluate_PIT_outlier_rate()

        try:
            y_uni = float(len(pit_vals)) / float(bins)
        except:
            y_uni = float(len(pit_vals)) / float(len(bins))
        if not show_qq:
            ax0.hist(pit_vals, bins=bins, alpha=0.7, label=label)
            ax0.set_ylabel("Number")
            ax0.hlines(y_uni, xmin=0, xmax=1, color="k")
            plt.ylim(
                0,
            )  # -0.001, 1.001)
        else:
            ax1 = ax0.twinx()
            ax1.hist(pit_vals, bins=bins, alpha=0.7)
            ax1.set_ylabel("Number")
            ax1.hlines(y_uni, xmin=0, xmax=1, color="k")
    leg = ax0.legend(handlelength=0, handletextpad=0, fancybox=True)
    for item in leg.legendHandles:
        item.set_visible(False)
    if show_qq:
        ax2 = plt.subplot(gs[1])
        ax2.plot(
            sample.qq[0],
            (sample.qq[1] - sample.qq[0]),
            c="r",
            linestyle="-",
            linewidth=3,
        )
        plt.ylabel("$\Delta$Q", fontsize=18)
        ax2.plot([0, 1], [0, 0], color="k", linestyle="--", linewidth=2)
        plt.xlim(-0.001, 1.001)
        plt.ylim(
            np.min([-0.12, np.min(sample.qq[1] - sample.qq[0]) * 1.05]),
            np.max([0.12, np.max(sample.qq[1] - sample.qq[0]) * 1.05]),
        )
    if show_pit:
        if show_qq:
            plt.xlabel("Q$_{theory}$ / PIT Value", fontsize=18)
        else:
            plt.xlabel("PIT Value", fontsize=18)
    else:
        if show_qq:
            plt.xlabel("Q$_{theory}$", fontsize=18)
    if savefig:
        fig_filename = str("plot_pit_qq_" + f"{(code).replace(' ', '_')}.png")
        plt.savefig(fig_filename)
    else:
        fig_filename = None

    return fig_filename

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
from qp import interp


zgrid = np.linspace(-1, 5, 200)
pdfs = res.objdata()["yvals"]
plot_pit_qq(
    pdfs,
    zgrid,
    ztrue,
    title="PIT-QQ - toy data",
    code="DeepDISC",
    pit_out_rate=pit_out_rate,
    savefig=False,
)

In [ ]:
res.objdata()